In [6]:
import pandas as pd
import re

# download_file(ANNOTATED_COMMENTS_URL, 'attack_annotated_comments.tsv')
comments = pd.read_csv('attack_annotated_comments.tsv', sep = '\t', index_col = 0, encoding='ISO-8859-1')
comments['comment'] = comments['comment'].apply(lambda x: x.replace("NEWLINE_TOKEN", " "))
comments['comment'] = comments['comment'].apply(lambda x: x.replace("TAB_TOKEN", " "))

# transform into lowercase
comments['comment'] = comments['comment'].apply(lambda x: " ".join(x.lower() for x in x.split()))

# remove punctuations
comments['comment'] = comments['comment'].str.replace('[^\w\s]','')

# remove emoji, references: https://gist.github.com/slowkow/7a7f61f495e3dbb7e3d767f97bd7304b
def remove_emoji(string):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', string)
comments['comment'] = comments['comment'].apply(lambda x: remove_emoji(x))

# remove whitespace
comments['comment'] = comments['comment'].apply(lambda x: x.strip())
print(comments['comment'].head(20))


rev_id
37675     this is not creative those are the dictionary ...
44816     the term standard model is itself less npov th...
49851     true or false the situation as of march 2002 w...
89320     next maybe you could work on being less condes...
93890                    this page will need disambiguation
102817    important note for all sysops there is a bug i...
103624    i removed the following all names of early pol...
111032    if you ever claimed in a judaic studies progra...
120283    my apologies im english i watch cricket i know...
128532    someone wrote more recognizable perhaps is a t...
133562    correct full biographical details will put dow...
138117    care should be taken to distinguish when and i...
155243    if i may butt in ive spent the last 14 hour fo...
177310    on my you will find the apology that i owe you...
192579    i fail to see the distinction who better than ...
201190                        gets far more tendentious yet
208009    as a person who has don

In [13]:
# create BoW model and train
# create the feature matrix
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline

pipe = Pipeline([
    ('count', CountVectorizer(stop_words='english', analyzer='word', ngram_range=(2, 2))),
    ('tfidf', TfidfTransformer(norm = 'l2')),
]).fit(comments['comment'])


vector = pipe['count'].transform(comments['comment'])
print(vector.shape)
print(pipe['tfidf'].idf_)

(115864, 2080357)
[11.96703382 11.96703382 11.96703382 ... 11.96703382 11.96703382
 11.96703382]
